In [25]:
import os 
import cv2
import torch
from torch.utils.data import DataLoader , Dataset 
from torchvision import transforms
import torch.nn as nn
from torch import optim

In [9]:
current_directory = os.getcwd()
os.chdir(current_directory)
datapath = r'Dataset\Pistachio_Image_Dataset\Pistachio_Image_Dataset'
subfolder = os.listdir(datapath)

images = []
labels = []
for sub in subfolder:
    label = subfolder.index(sub)
    path = os.path.join(datapath,sub)
    for img_path in os.listdir(path):
        image = cv2.imread(os.path.join(path,img_path))
        images.append(image)
        labels.append(label)

In [19]:
class Dataprep(Dataset):
    def __init__(self, features, labels, transforms = None):
        self.features = features
        self.labels = labels
        self.transforms = transforms
    
    def __getitem__(self, index) :
        image = self.features[index]
        label = self.labels[index]
        if self.transforms:
            image = self.transforms(image)

        return image, label
    
    def __len__(self):
        return len(self.labels)


data_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((200,200))])
dataset = Dataprep(images,labels,data_transforms)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
# data_sample = next(iter(data_loader))

In [30]:
class Pistachiomodel(nn.Module):
    def __init__(self):
        super(Pistachiomodel,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64,kernel_size=2,stride=2,padding=0)
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(p=0.5)
        self.linear = nn.Linear(64*50*50,2)

    def forward(self,data):
        data = self.conv1(data)
        data = self.maxpool(data)
        data = self.activation(data)
        data = self.flatten(data)
        data = self.dropout(data)
        data = self.linear(data)

        return data
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Pistachiomodel().to(device)
model.eval()

Pistachiomodel(
  (conv1): Conv2d(3, 64, kernel_size=(2, 2), stride=(2, 2))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (activation): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=160000, out_features=2, bias=True)
)

In [31]:
learning_rate = 0.001
epochs = 100
optimizer = optim.SGD(model.parameters(),learning_rate)
criterion = nn.CrossEntropyLoss()

total_sample = 0
total_correct = 0
for i in range(epochs):
    for image,target in data_loader:
        image = image.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(image)

        _, predicted = torch.max(output, 1)
        total_sample += target.size(0)
        total_correct += (predicted == target).sum().item()

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    accuracy = total_correct / total_sample
    print(f'Epochs: {i+1} out of {epochs} || Loss: {loss.item()} || Accuracy: {accuracy}')

Epochs: 1 out of 100 || Loss: 0.7237868905067444 || Accuracy: 0.6918063314711359
Epochs: 2 out of 100 || Loss: 0.09502732753753662 || Accuracy: 0.7374301675977654
Epochs: 3 out of 100 || Loss: 0.37299448251724243 || Accuracy: 0.7613283674736189
Epochs: 4 out of 100 || Loss: 0.8518340587615967 || Accuracy: 0.7775837988826816
Epochs: 5 out of 100 || Loss: 0.11370278149843216 || Accuracy: 0.7864990689013035
Epochs: 6 out of 100 || Loss: 0.14258450269699097 || Accuracy: 0.7952358783364369
Epochs: 7 out of 100 || Loss: 0.05550546199083328 || Accuracy: 0.8021415270018621
Epochs: 8 out of 100 || Loss: 0.18157942593097687 || Accuracy: 0.8074953445065177
Epochs: 9 out of 100 || Loss: 0.5549629926681519 || Accuracy: 0.8122284295468652
Epochs: 10 out of 100 || Loss: 0.21372802555561066 || Accuracy: 0.8157355679702049
Epochs: 11 out of 100 || Loss: 0.5344483852386475 || Accuracy: 0.819239884882343
Epochs: 12 out of 100 || Loss: 0.056300655007362366 || Accuracy: 0.8224705152079453
Epochs: 13 out of